# 과적합 피하기 - 초음파광물 사례

In [64]:
# tensorboard init
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir() #예를 들면, './my_logs/run_2019_06_07-15_15_22'

In [65]:
import pandas as pd

df = pd.read_csv('../dataset/sonar.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [66]:
import numpy as np
import tensorflow as tf
import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

In [67]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [68]:
# 데이터 입력
dataset = df.values
X = np.array(dataset[:,0:-1], dtype=np.float64)
Y_obj = dataset[:,-1]

In [69]:
type(Y_obj)

numpy.ndarray

In [70]:
# 문자열 변환
e = LabelEncoder().fit(Y_obj)
Y = np.array(e.transform(Y_obj), dtype=np.float64)

In [71]:
Y_obj[:5], Y[:5]

(array(['R', 'R', 'R', 'R', 'R'], dtype=object), array([1., 1., 1., 1., 1.]))

In [72]:
# 80% Train set, 20% Test set으로 분리 (+ 20% validation set)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=156)

In [73]:
# 모델 설정
model = Sequential([
    Dense(24, input_dim=60, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 24)                1464      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                250       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 1,725
Trainable params: 1,725
Non-trainable params: 0
_________________________________________________________________


In [74]:
# 모델 컴파일 
model.compile(loss='binary_crossentropy',   # mean_squared_error
              optimizer='adam',
              metrics=['accuracy'])

In [75]:
# 모델 실행
import keras
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
model.fit(X, Y, epochs=200, batch_size=5, callbacks=[tensorboard_cb], validation_split=0.2)

Train on 166 samples, validate on 42 samples
Epoch 1/200
166/166 [==============================] - 1s 3ms/sample - loss: 0.6742 - accuracy: 0.6386 - val_loss: 0.7960 - val_accuracy: 0.0952
Epoch 2/200
166/166 [==============================] - 0s 602us/sample - loss: 0.6466 - accuracy: 0.6928 - val_loss: 0.8031 - val_accuracy: 0.2143
Epoch 3/200
166/166 [==============================] - 0s 542us/sample - loss: 0.6177 - accuracy: 0.6867 - val_loss: 0.7880 - val_accuracy: 0.2857
Epoch 4/200
166/166 [==============================] - 0s 706us/sample - loss: 0.5966 - accuracy: 0.6928 - val_loss: 0.8543 - val_accuracy: 0.2381
Epoch 5/200
166/166 [==============================] - 0s 749us/sample - loss: 0.5754 - accuracy: 0.7349 - val_loss: 0.7769 - val_accuracy: 0.4524
Epoch 6/200
166/166 [==============================] - 0s 458us/sample - loss: 0.5569 - accuracy: 0.7349 - val_loss: 0.8721 - val_accuracy: 0.3095
Epoch 7/200
166/166 [==============================] - 0s 487us/sample - lo

Epoch 56/200
166/166 [==============================] - 0s 483us/sample - loss: 0.1902 - accuracy: 0.9398 - val_loss: 1.5202 - val_accuracy: 0.5000
Epoch 57/200
166/166 [==============================] - 0s 420us/sample - loss: 0.1972 - accuracy: 0.9458 - val_loss: 1.1537 - val_accuracy: 0.5952
Epoch 58/200
166/166 [==============================] - 0s 603us/sample - loss: 0.1811 - accuracy: 0.9639 - val_loss: 1.4966 - val_accuracy: 0.5000
Epoch 59/200
166/166 [==============================] - 0s 642us/sample - loss: 0.1814 - accuracy: 0.9518 - val_loss: 1.3648 - val_accuracy: 0.5476
Epoch 60/200
166/166 [==============================] - 0s 555us/sample - loss: 0.1731 - accuracy: 0.9518 - val_loss: 1.1297 - val_accuracy: 0.6190
Epoch 61/200
166/166 [==============================] - 0s 646us/sample - loss: 0.1694 - accuracy: 0.9578 - val_loss: 1.1559 - val_accuracy: 0.6190
Epoch 62/200
166/166 [==============================] - 0s 643us/sample - loss: 0.1750 - accuracy: 0.9639 - val_

166/166 [==============================] - 0s 608us/sample - loss: 0.0479 - accuracy: 0.9940 - val_loss: 1.6993 - val_accuracy: 0.7381
Epoch 112/200
166/166 [==============================] - 0s 432us/sample - loss: 0.0407 - accuracy: 0.9940 - val_loss: 2.2342 - val_accuracy: 0.5476
Epoch 113/200
166/166 [==============================] - 0s 484us/sample - loss: 0.0474 - accuracy: 1.0000 - val_loss: 1.9911 - val_accuracy: 0.6190
Epoch 114/200
166/166 [==============================] - 0s 422us/sample - loss: 0.0414 - accuracy: 0.9880 - val_loss: 2.0076 - val_accuracy: 0.6190
Epoch 115/200
166/166 [==============================] - 0s 482us/sample - loss: 0.0387 - accuracy: 1.0000 - val_loss: 2.0112 - val_accuracy: 0.6190
Epoch 116/200
166/166 [==============================] - 0s 422us/sample - loss: 0.0383 - accuracy: 1.0000 - val_loss: 1.9545 - val_accuracy: 0.6667
Epoch 117/200
166/166 [==============================] - 0s 481us/sample - loss: 0.0365 - accuracy: 1.0000 - val_loss: 1

166/166 [==============================] - 0s 543us/sample - loss: 0.0082 - accuracy: 1.0000 - val_loss: 2.6558 - val_accuracy: 0.6429
Epoch 167/200
166/166 [==============================] - 0s 460us/sample - loss: 0.0088 - accuracy: 1.0000 - val_loss: 2.3877 - val_accuracy: 0.7619
Epoch 168/200
166/166 [==============================] - 0s 497us/sample - loss: 0.0087 - accuracy: 1.0000 - val_loss: 2.3961 - val_accuracy: 0.7619
Epoch 169/200
166/166 [==============================] - 0s 482us/sample - loss: 0.0081 - accuracy: 1.0000 - val_loss: 2.8127 - val_accuracy: 0.5952
Epoch 170/200
166/166 [==============================] - 0s 435us/sample - loss: 0.0092 - accuracy: 1.0000 - val_loss: 2.6193 - val_accuracy: 0.7381
Epoch 171/200
166/166 [==============================] - ETA: 0s - loss: 0.0296 - accuracy: 1.00 - 0s 591us/sample - loss: 0.0077 - accuracy: 1.0000 - val_loss: 2.5723 - val_accuracy: 0.7381
Epoch 172/200
166/166 [==============================] - 0s 425us/sample - los

In [55]:
# 모델을 컴퓨터에 저장
model.save('model/my_model.h5')

In [56]:
del model # 테스트를 위해 메모리 내의 모델을 삭제

In [57]:
model.summary()

NameError: name 'model' is not defined

In [61]:
from keras.models import load_model
model = load_model('model/my_model.h5') # 모델을 새로 불러옴

ValueError: Unknown initializer: GlorotUniform

In [59]:
# 결과 출력 
print("\n Accuracy: %.4f" % (model.evaluate(X, Y, verbose=2)[1]))

NameError: name 'model' is not defined

In [ ]:
# 테스트셋에 모델 적용
print("n Test Accuracy: %.4f" % ( model.evaluate( X_test , y_test ) [1]))